In [1]:
import tensorflow as tf
import numpy as np

/Users/kiizuka/.venv/py3-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kiizuka/.venv/py3-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kiizuka/.venv/py3-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kiizuka/.venv/py3-tf/lib/python3.7/site-packages/tensorflow/

## Check group convolution using TensorFlow

In [21]:
# Initialize input data 
conv_input = np.arange(5*5*4, dtype=np.float32)
conv_input = conv_input.reshape([1, 4, 5, 5])
tf_input = conv_input.transpose([0, 2, 3, 1]) # NCHW to NHWC
print(tf_input.shape)
weights = np.arange(3*3*2*6, dtype=np.float32)
weights = weights.reshape([6, 2, 3, 3])
# tf_weights = weights.transpose([2, 3, 1, 0]) # NCHW to NHWC
tf_weights = weights.transpose([2, 3, 1, 0]) # NCHW to NHWC
# print(tf_weights)
bias = np.zeros([6,])

(1, 5, 5, 4)


In [22]:
def conv(x, weights, biases, stride_y, stride_x, name,
         padding='SAME', groups=1):

    # Create lambda function for the convolution
    convolve = lambda i, k: tf.nn.conv2d(i, k,
                                         strides=[1, stride_y, stride_x, 1],
                                         padding=padding)

    if groups == 1:
        conv = convolve(x, weights)

    # In the cases of multiple groups, split inputs & weights and
    else:
        # Split input and weights and convolve them separately
        input_groups = tf.split(axis=3, num_or_size_splits=groups, value=x)
        weight_groups = tf.split(axis=3, num_or_size_splits=groups,
                                 value=weights)
        output_groups = [convolve(i, k) for i, k in zip(input_groups, weight_groups)]

        # Concat the convolved output together again
        conv = tf.concat(axis=3, values=output_groups)

    # Add biases
    bias = tf.reshape(tf.nn.bias_add(conv, biases), tf.shape(conv))
    return bias

In [26]:
stride = 1
padding = 0
output = conv(tf_input, tf_weights, bias, 1, 1, padding='VALID', name='group_test_conv', groups=2)
with tf.compat.v1.Session() as sess:
    out = sess.run(output)
    print(type(out))
    print(out.shape)
    result = out.transpose([3, 0, 1, 2])
    print(result)

<class 'numpy.ndarray'>
(1, 3, 3, 6)
[[[[  4035.   4188.   4341.]
   [  4800.   4953.   5106.]
   [  5565.   5718.   5871.]]]


 [[[ 10029.  10506.  10983.]
   [ 12414.  12891.  13368.]
   [ 14799.  15276.  15753.]]]


 [[[ 16023.  16824.  17625.]
   [ 20028.  20829.  21630.]
   [ 24033.  24834.  25635.]]]


 [[[ 78267.  79392.  80517.]
   [ 83892.  85017.  86142.]
   [ 89517.  90642.  91767.]]]


 [[[100461. 101910. 103359.]
   [107706. 109155. 110604.]
   [114951. 116400. 117849.]]]


 [[[122655. 124428. 126201.]
   [131520. 133293. 135066.]
   [140385. 142158. 143931.]]]]


In [2]:
conv_input = np.ones([1,  227, 227, 3])
weights = np.ones([11, 11, 3, 55])
stride = 4
padding = 0
output = tf.nn.conv2d(
    conv_input,
    filter=weights,
    strides=stride,
    padding='VALID',
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name='test_conv'
)
with tf.Session() as sess:
    out = sess.run(output)

# Check LRN(Local Responsive Normalization)

In [10]:
# params for lrn
depth_radius = 2
bias = 2.0
alpha = 0.0001
beta = 0.75

# params for max_pool
ksize = 3
padding = 0
strides = 2

pool1_input = np.ones([1, 55, 55, 96], dtype=np.float32) # NHWC format
# print(pool1_input.dtype)

relu = tf.nn.relu(pool1_input)
lrn = tf.nn.lrn(relu, depth_radius, bias=bias, alpha=alpha, beta=beta)
print(lrn.shape)

# pool_test = np.ones([1, 55, 55, 96], dtype=np.float32)
# pool_out = tf.nn.max_pool2d(pool_test, ksize=ksize, strides=strides, padding='VALID', data_format='NHWC', name='pool1')

with tf.compat.v1.Session() as sess:
    out = sess.run(lrn)
    trans_out = out.transpose([0, 3, 1, 2]) # NHWC -> NCHW
    for i in range(trans_out.shape[1]):
        print(trans_out[0][i][0][0])

(1, 55, 55, 96)
0.59453666
0.59451437
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.5944921
0.59451437
0.59453666
